In [1]:
%load_ext autoreload
%autoreload 2

In [15]:
from rlway.pyosrd import OSRD

In [25]:
OSRD.use_cases

['cvg_dvg_bug', 'cvg_dvg_no_link', 'divergence', 'station_capacity2']

In [27]:
sim = OSRD(use_case='station_capacity2', dir='station_capacity2')

[19:05:52,233] [INFO]  [StandaloneSimulationCommand] Loading infra: station_capacity2/infra.json
[19:05:52,413] [INFO]              [FullInfra] parsing infra
[19:05:53,798] [INFO]              [FullInfra] adaptation to kotlin
[19:05:53,991] [INFO]              [FullInfra] loading signals
[19:05:54,041] [INFO]              [FullInfra] building blocks
[19:05:54,085] [INFO]  [StandaloneSimulationCommand] Loading input: station_capacity2/simulation.json
[19:05:54,317] [INFO]  [StandaloneSimulationCommand] Running simulation for schedule group: group.18
[19:05:54,685] [INFO]  [StandaloneSimulationCommand] Running simulation for schedule group: group.19
[19:05:54,722] [INFO]  [StandaloneSimulationCommand] All simulations completed
[19:05:54,723] [INFO]  [StandaloneSimulationCommand] Saving results at: station_capacity2/results.json


In [28]:
sim.draw_infra()

In [22]:
import rlway
help(rlway.pyosrd.use_cases.cvg_dvg)

Help on module rlway.pyosrd.use_cases.cvg_dvg in rlway.pyosrd.use_cases:

NAME
    rlway.pyosrd.use_cases.cvg_dvg

FUNCTIONS
    cvg_dvg(dir: str, infra_json: str = 'infra.json', simulation_json: str = 'simulation.json') -> None
        station0 (2 tracks)                        station1 (2 tracks)
        
               ┎S0                                      S4┐
        (T0)-----D0-                                  --D4---------(T4)-->
                    \   S2┐                    ┎S3  /
                   CVG>-D2-----(T2)--+--(T3)----D3-<DVG
               ┎S1   /                              \   S5┐
        (T1)-----D1-                                  --D5---------(T5)-->
        
        All tracks are 500m long
        Train 0 starts from T0 at t=0 and arrives at T4
        Train 1 starts from T1 at t=100 and arrives at T5

FILE
    /home/renan/regul/src/rlway/pyosrd/use_cases/cvg_dvg.py


